In [ ]:
#required installation for map vizualiztion
using Pkg
Pkg.add("PyCall")
Pkg.add("Conda")
Pkg.add("OpenStreetMapX")
using Conda
Conda.runconda(`install folium -c conda-forge -y`)

Let us load the map of east of Reno (this file is provided together with the `OpenStreetMapX.jl` package)  and move few thousands of cars around it between random pairs of destinations

In [59]:
using OpenStreetMapX
# you can replace the line below with a String representing another path to an OSM map file.
map_file_path = joinpath(dirname(pathof(OpenStreetMapX)),"..","test/data/reno_east3.osm")
mx = get_map_data(map_file_path, use_cache=false);
using Random 
Random.seed!(0)
node_ids = collect(keys(mx.intersections));

const N_AMB = 5 #number of ambulances

ambulance_loc = rand(node_ids, N_AMB)
amb_ranges_nodes = Vector{Vector{Int}}(undef, N_AMB)
amb_ranges_times = Vector{Vector{Float64}}(undef, N_AMB)

for i in 1:N_AMB
    amb_ranges_nodes[i], amb_ranges_times[i] = OpenStreetMapX.nodes_within_driving_time(mx, [ambulance_loc[i]],100.0)
end

using Statistics
function estimate_amb_layout_goodness(ambulance_loc:: Vector{Int}, mx::MapData)
    ranges_nodes, ranges_times = OpenStreetMapX.nodes_within_driving_time(mx, ambulance_loc,100000.0)
    #todo - consult with Bartosz but this should return the average travel time from all reachable nodes    
    mean(ranges_times)
end



estimate_amb_layout_goodness (generic function with 1 method)

In [61]:
using PyCall
flm = pyimport("folium")
matplotlib_cm = pyimport("matplotlib.cm")
matplotlib_colors = pyimport("matplotlib.colors")

cmap = matplotlib_cm.get_cmap("prism")

m = flm.Map()
for i in 1:N_AMB
    loc = LLA(mx.nodes[ambulance_loc[i]],mx.bounds)
    
    info = "Ambulance $i\n<BR>"*
        "Node $(ambulance_loc[i])\n<br>" *
        "Loc: $(round.((loc.lat, loc.lon),digits=4))\n<br>";
    flm.Rectangle(        
        [(loc.lat-0.0004, loc.lon-0.0004),(loc.lat+0.0004, loc.lon+0.0004)],
        popup=info,
        tooltip=info,
        color="black"
    ).add_to(m)
    
    for k=1:length(amb_ranges_nodes[i])
        info = "Ambulance $i\n<BR>"*
        "Node $(amb_ranges_nodes[i][k])\n<br>" *
        "Travel time $(round(amb_ranges_times[i][k],digits=1)))"
        node_loc = LLA(mx.nodes[amb_ranges_nodes[i][k]],mx.bounds)
        
        flm.Circle(        
        (node_loc.lat, node_loc.lon),
        popup=info,
        tooltip=info,
        color=matplotlib_colors.to_hex(cmap(amb_ranges_times[i][k]/300)),
        weight=0.5,
        #fill=true,
        #fill_color="crimson",
        radius = 50
        
    ).add_to(m)
    end 
    
end

MAP_BOUNDS = [(mx.bounds.min_y,mx.bounds.min_x),(mx.bounds.max_y,mx.bounds.max_x)]
flm.Rectangle(MAP_BOUNDS, color="black",weight=1).add_to(m)
m.fit_bounds(MAP_BOUNDS)
m

PyObject <folium.folium.Map object at 0x000000003CC9E2E8>